## I) Pré traitement

### 0) Récupération

In [ ]:
db = load_all_sentences();
print('chargement de {} vers dans la db'.format(len(db.keys())))

In [ ]:
from collections import defaultdict
base_artistes = defaultdict(set)
for k,v in db.iteritems():
    base_artistes[v['artistes']].add(k)
artistes = { k:v for k,v in artistes.iteritems() if len(v) > 200 }
print('{} artistes'.format(len(artistes)))

### 1) Exploration

In [ ]:
import nltk
test = "Bonjour, je suis un texte d'exemple pour le cours d'Openclassrooms. Soyez attentifs à ce cours !"

nltk.word_tokenize(test)

#### Tokenization

In [ ]:
tokenizer = nltk.RegexpTokenizer(r'\w+')
tokenizer.tokenize("Bonjour, je suis un texte d'exemple pour le cours d'Openclassrooms. Soyez attentifs à ce cours !")


In [ ]:
tokenizer = nltk.RegexpTokenizer(r'\w+')

def freq_stats_corpora():
    corpora = defaultdict(list)

    # Création d'un corpus de tokens par artiste
    for artiste,sentence_id in artistes.iteritems():
        for sentence_id in sentence_id:
            corpora[artiste] += tokenizer.tokenize(
                db[sentence_id]['text'].decode('utf-8').lower()
            )

    stats, freq = dict(), dict()

    for k, v in corpora.iteritems():
        freq[k] = fq = nltk.FreqDist(v)
        stats[k] = {'total': len(v)}

    return (freq, stats, corpora)

# Récupération des comptages
freq, stats, corpora = freq_stats_corpora()
df = pd.DataFrame.from_dict(stats, orient='index')

# Affichage des fréquences
df.sort(columns='total', ascending=False)
df.plot(kind='bar', color="#f56900", title='Top 50 Rappeurs par nombre de mots')



In [ ]:
def freq_stats_corpora():
    corpora = defaultdict(list)
    for artiste,sentence_ids in artistes.iteritems():
        for sentence_id in sentence_ids:
            corpora[artiste] += tokenizer.tokenize(
                db[sentence_id]['text'].decode('utf-8').lower()
            )

    stats, freq = dict(), dict()

    for k, v in corpora.iteritems():
        freq[k] = fq = nltk.FreqDist(v)
        stats[k] = {'total': len(v), 'unique': len(fq.keys())}

    return (freq, stats, corpora)


### 2) Nettoyage / Normalisation

#### 1) supprimer les stopwords

In [ ]:
# Premièrement, on récupère la fréquence totale de chaque mot sur tout le corpus d'artistes
freq_totale = nltk.Counter()
for k, v in corpora.iteritems():
    freq_totale += freq[k]

# Deuxièmement on décide manière un peu arbitraire du nombre de mots les plus fréquents à supprimer. On pourrait afficher un graphe d'évolution du nombre de mots pour se rendre compte et avoir une meilleure heuristique.
most_freq = zip(*freq2.most_common(100))[0] # stopwords =

# On créé notre set de stopwords final qui cumule ainsi les 100 mots les plus fréquents du corpus ainsi que l'ensemble de stopwords par défaut présent dans la librairie NLTK
sw = set()
sw.update(stopwords)
sw.update(tuple(nltk.corpus.stopwords.words('french')))


In [ ]:
def freq_stats_corpora2(lookup_table=[]):
    corpora = defaultdict(list)
    for artist, block_ids in lt_artists.iteritems():
        for block_id in block_ids:
            tokens = tokenizer.tokenize(db_flat[block_id]['text'].decode('utf-8'))
            corpora[artist] += [w for w in tokens if not w in list(sw)]

    stats, freq = dict(), dict()

    for k, v in corpora.iteritems():
        freq[k] = fq = nltk.FreqDist(v)
        stats[k] = {'total': len(v), 'unique': len(fq.keys())}
    return (freq, stats, corpora)

freq2, stats2, corpora2 = freq_stats_corpora2()


#### 2) Lemmatisation ou racinisation (stemming)

##### Racinisation :

In [ ]:
from nltk.stem.snowball import FrenchStemmer

stemmer = FrenchStemmer()

def freq_stats_corpora3(lookup_table=[]):
    corpora = defaultdict(list)
    for artist, block_ids in lt_artists.iteritems():
        for block_id in block_ids:
            tokens = tokenizer.tokenize(db_flat[block_id]['text'].decode('utf-8').lower())
            corpora[artist] += [stemmer.stem(w) for w in tokens if not w in list(sw)]

    stats, freq = dict(), dict()

    for k, v in corpora.iteritems():
        freq[k] = fq = nltk.FreqDist(v)
        stats[k] = {'total': len(v), 'unique': len(fq.keys())}
    return (freq, stats, corpora)

freq3, stats3, corpora3 = freq_stats_corpora3()
df3 = pd.DataFrame.from_dict(stats3, orient='index').sort(columns='unique', ascending=False)


### TP

#### 1) Exploratory Data Analysis

Quelle est la forme du Dataframe ?

Y a t-il des valeurs manquantes ou des valeurs dupliquées ?

Quelles sont les colonnes qui vont nous intéresser ?

Y a-t-il des données aberrantes ou des incohérences majeures dans les données ?

Y a t-il des tweets anormalement longs / courts ? Peut-on les considérer comme des outliers ?

Quel est le ratio tweet qui parlent de “catastrophes” / tweet normaux ?

En regardant quelques tweets au hasard, peut-on deviner facilement la “target” ?

Peut-on déjà détecter des “patterns” ou des mots clés dans les tweets?

A votre avis quel serait l’accuracy score qu’un humain pourrait obtenir s’il prédisait  les données “à la main” ?

#### 2) Text Processing

Pouvez-vous écrire une fonction qui : tokenize un document, supprime les stopwords, supprime les tokens de moins de 3 lettres ?

Comment peut-on reconstituer le corpus (c'est-à dire un texte avec l’ensemble des documents) ?

Une fois ce corpus constitué, combien de tokens uniques le constitue? Ce nombre vous apparaît-il faible, important, gigantesque ?

Comment réduire ce nombre de tokens uniques, ou autrement dit “comment réduire la taille du vocabulaire” de ce corpus ?

Combien de tokens sont présents une seule fois ? Ces tokens nous seront-ils utiles ?

Appliquer une méthode de stemmatisation ou de lemmatisation peut-elle nous aider à réduire la dimensionnalité du corpus ?

Comment visualiser graphiquement, par un WordCloud par exemple, les tokens les plus présents ?

Pouvez vous appliquer tous les traitements évoqués afin de créer une nouvelle colonne “text” qui serait plus pertinente ?

# II) Transformations

#### 1) Bag of Words / n gram / TF-IDF

In [ ]:
import os
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

for subdir, dirs, files in os.walk(path):
    for file in files:
        file_path = subdir + os.path.sep + file
        shakes = open(file_path, 'r')
        text = shakes.read()
        lowers = text.lower()
        no_punctuation = lowers.translate(None, string.punctuation)
        token_dict[file] = no_punctuation

tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words=sw)
values = tfidf.fit_transform(token_dict.values())

La matrice TF-IDF est définie pour chaque mot relativement à un corpus, comme le produit TF * IDF où:

TF = nombre de fois où le mot est dans le document / nombre de mots dans le document
IDF = nombre de documents / nombre de documents où apparaît le mot

#### 2) Words embeddings : Word2vec

#### 3) LDA

##### Préttt

In [1]:
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

In [3]:
from sklearn.decomposition import LatentDirichletAllocation
n_topics = 20

In [5]:
# TF : term frequency

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)

##### Créer le modèle LDA

In [6]:
# Créer le modèle LDA
lda = LatentDirichletAllocation(
        n_components=n_topics,
        max_iter=5,
        learning_method='online',
        learning_offset=50.,
        random_state=0)

# Fitter sur les données
lda.fit(tf)

LatentDirichletAllocation(learning_method='online', learning_offset=50.0,
                          max_iter=5, n_components=20, random_state=0)

In [15]:
tf_feature_names = tf_vectorizer.get_feature_names_out()
tf_feature_names

array(['00', '000', '01', '02', '03', '04', '0d', '0t', '10', '100', '11',
       '12', '128', '13', '14', '145', '15', '16', '17', '18', '19',
       '1990', '1991', '1992', '1993', '1d9', '1st', '1t', '20', '200',
       '21', '22', '23', '24', '25', '250', '26', '27', '28', '29', '2di',
       '2tm', '30', '300', '31', '32', '33', '34', '34u', '35', '36',
       '37', '38', '39', '3d', '3t', '40', '42', '43', '44', '45', '50',
       '500', '55', '60', '64', '6ei', '70', '75', '75u', '7ey', '7u',
       '80', '800', '86', '90', '91', '92', '93', '9v', 'a86', 'able',
       'ac', 'accept', 'access', 'according', 'act', 'action', 'actually',
       'add', 'addition', 'address', 'administration', 'advance', 'age',
       'ago', 'agree', 'ah', 'air', 'al', 'algorithm', 'allow', 'allowed',
       'alt', 'america', 'american', 'analysis', 'anonymous', 'answer',
       'answers', 'anti', 'anybody', 'apparently', 'appears', 'apple',
       'application', 'applications', 'appreciate', 'appre

##### Evaluation

In [16]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic {}:".format(topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
display_topics(lda, tf_feature_names, no_top_words)


Topic 0:
people gun state control right guns crime states law police
Topic 1:
time question book years did like don space answer just
Topic 2:
mr line rules science stephanopoulos title current define int yes
Topic 3:
key chip keys clipper encryption number des algorithm use bit
Topic 4:
edu com cs vs w7 cx mail uk 17 send
Topic 5:
use does window problem way used point different case value
Topic 6:
windows thanks know help db does dos problem like using
Topic 7:
bike water effect road design media dod paper like turn
Topic 8:
don just like think know people good ve going say
Topic 9:
car new price good power used air sale offer ground
Topic 10:
file available program edu ftp information files use image version
Topic 11:
ax max b8f g9v a86 145 pl 1d9 0t 34u
Topic 12:
government law privacy security legal encryption court fbi technology information
Topic 13:
card bit memory output video color data mode monitor 16
Topic 14:
drive scsi disk mac hard apple drives controller software port
T

#### Une alternative, NMF


Une autre type de modélisation de sujet automatique non supervisée est NMF (Negative Matrix Factorisation).

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF

no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95,
                                   min_df=2,
                                   max_features=no_features,
                                   stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

no_topics = 20

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd')
nmf.fit(tfidf)

no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)


C:\Users\oumei\AppData\Roaming\Python\Python39\site-packages\sklearn\decomposition\_nmf.py:1477: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  warnings.warn(


Topic 0:
people time right did good said say make way government
Topic 1:
window problem using server application screen display motif manager running
Topic 2:
god jesus bible christ faith believe christian christians sin church
Topic 3:
game team year games season players play hockey win league
Topic 4:
new 00 sale 10 price offer shipping condition 20 15
Topic 5:
thanks mail advance hi looking info help information address appreciated
Topic 6:
windows file files dos program version ftp ms directory running
Topic 7:
edu soon cs university ftp internet article email pub david
Topic 8:
key chip clipper encryption keys escrow government public algorithm nsa
Topic 9:
drive scsi drives hard disk ide floppy controller cd mac
Topic 10:
just ll thought tell oh little fine work wanted mean
Topic 11:
does know anybody mean work say doesn help exist program
Topic 12:
card video monitor cards drivers bus vga driver color memory
Topic 13:
like sounds looks look bike sound lot things really thing
To